Program to generate spectrograms from an acoustic waveform.

In [ ]:
from pathlib import Path, PurePath

import numpy

from scipy import signal
import matplotlib.pyplot as plt

try:
  from pydub import AudioSegment
  from pydub.utils import get_array_type
except:
  !pip3 install pydub

try:
  import csv
except:
  !pip3 install csv

In [ ]:
# Path for directory containing raw audio clips
raw_audio_clips_path = 'https://drive.google.com/drive/folders/1BIRX0Yb1JVuvmby-JZwHIpbiAc6lCcQv?usp=share_link'

# Path for directory containing spectrograms generated from raw audio clips
raw_audio_spectrograms_path = 'https://drive.google.com/drive/folders/1FFrMgj5Zg6ZNtK6ff7dmy1aPd5IABXLJ?usp=share_link'

In [ ]:
# Create an AudioSegment instance from the audio file at the path specified.
# The information in the audio file is then used to create a numpy array, which
# is used as an argument in scipy.signal.spectrogram() to generate the
# corresponding spectrogram.

def generate_spectrogram_from_audio(path, stem):

  instance_AS = AudioSegment.from_file(path)
  samplerate = instance_AS.frame_rate
  samples = numpy.array([])

  bit_depth = instance_AS.sample_width*8
  array_type = get_array_type(bit_depth)
  data_instance_AS = numpy.fromstring(instance_AS._data, array_type)

  channel_samples = []
  for channel in numpy.arange(instance_AS.channels):
    channel_samples.append(data_instance_AS[channel::instance_AS.channels])
  samples = numpy.array(channel_samples, dtype=array_type)

  frequencies, times, Sxx = signal.spectrogram(data_instance_AS, samplerate, scaling='spectrum')

  plt.pcolormesh(times, frequencies, numpy.log10(Sxx[0]), cmap='jet')
  plt.xticks(numpy.arange(0, 10, step=.2), rotation = (90))
  plt.tick_params(axis='x', which='major', labelsize=7)
  
  plt.ylabel('Frequency [Hz]')
  plt.xlabel('Time [sec]')
  
  plt.savefig(Path(raw_audio_spectrograms_path, stem +'.png'), format='png')

for path in Path(raw_audio_clips_path).rglob('*.flac'):
  generate_spectrogram_from_audio(path, PurePath(path).stem)
for path in Path(raw_audio_clips_path).rglob('*.WAV'):
  generate_spectrogram_from_audio(path, PurePath(path).stem)

#Alternately, can use the following in place of the two for loops above.
#for path in Path(raw_audio_clips_path).glob('*.[fW][lA][aV]*'):
  #generate_spectrogram_from_audio(path, PurePath(path).stem)